# Exploring a new features 

In [433]:
import uproot4
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import pandas as pd
import awkward1


ccup9_2015 = r'D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2015\4Prongs2015o.root'

events = uproot4.open(ccup9_2015)['4Prongs/events']
events.show()


name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
RunNum               | int32_t                  | AsDtype('>i4')
PeriodNumber         | uint32_t                 | AsDtype('>u4')
OrbitNumber          | uint32_t                 | AsDtype('>u4')
BunchCrossNumber     | uint16_t                 | AsDtype('>u2')
Mass                 | float                    | AsDtype('>f4')
Pt                   | float                    | AsDtype('>f4')
Q                    | int16_t                  | AsDtype('>i2')
Rapidity             | float                    | AsDtype('>f4')
Phi                  | float                    | AsDtype('>f4')
ZNAenergy            | float                    | AsDtype('>f4')
ZNCenergy            | float                    | AsDtype('>f4')
ZPAenergy            | float                    | AsDtype('>f4')
ZPCenergy            | float                    | AsDtype(

In [427]:


TPCMask = (np.abs(dfs4Tracks.T_NumberOfSigmaTPCPion) < 3) & (dfs4Tracks.T_TPCNCls > 50) & (dfs4Tracks.T_TPCRefit == 1)

plt.style.use(hep.style.ROOT)
fig = plt.figure(figsize=(15,7))
# ax = fig.add_axes([0,0,1,1])
ax = fig.add_subplot()

fig.suptitle(f'4track events pt', fontsize=32)
tpcimpPt = []
colors = ['red','green', 'yellow', 'orange', 'black']
labels = ['ITS & (>= 0TPC)','ITS & (>= 1TPC)', 'ITS & (>= 2TPC)', 'ITS & (>= 3TPC)', 'ITS & (= 4TPC)']


for i in range(5):
    ITSnTPCEvents = pd.unique(dfs4Tracks.reset_index().entry)[TPCMask.groupby('entry').sum() >= i]
    ITSnTPCTracks = dfs4Tracks.loc[ITSnTPCEvents]
    tmpPt = np.sqrt(ITSnTPCTracks.groupby('entry').T_Px.sum()**2 + ITSnTPCTracks.groupby('entry').T_Py.sum()**2)
    tpcimpPt.append(tmpPt)
    ax.hist(tpcimpPt[i], bins=100, range=(0,2), histtype='step', color=colors[i], linewidth=2, label=labels[i]+f';{(tmpPt<0.15).sum()}')

ax.set_xlabel('$p_t, GeV$')
ax.set_ylabel('# events')

ax.add_patch(Rectangle((0,0.15),0.15,900,fc='lightgrey',alpha=0.4))

ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …